# Interbank Markets Analysis - Kenya's Money Market Operations
## Deep Dive into Central Bank Monetary Policy and Market Liquidity

This notebook analyzes:
- Interbank lending rates and volumes
- Central Bank Rate (CBR) impact analysis
- Repo and reverse repo operations
- Discount window usage patterns
- Money market liquidity conditions
- Government securities market dynamics

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Advanced analytics libraries
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import networkx as nx

In [ ]:
class InterbankMarketsAnalyzer:
    """Advanced Interbank Markets and Monetary Policy Analysis Engine"""
    
    def __init__(self):
        self.interbank_data = {}
        self.cbr_data = None
        self.repo_data = None
        self.securities_data = None
        
    def load_interbank_data(self, data_path="data/raw/"):
        """Load all interbank market datasets"""
        
        datasets_loaded = 0
        
        try:
            # Interbank rates and volumes
            self.interbank_rates = pd.read_csv(f"{data_path}Interbank Rates  Volumes.csv", skiprows=2)
            self.interbank_rates_alt = pd.read_csv(f"{data_path}Interbank Rates () .csv", skiprows=2)
            datasets_loaded += 2
            print("✅ Loaded Interbank Rates Data")
        except Exception as e:
            print(f"❌ Could not load Interbank Rates: {e}")
            
        try:
            # Central Bank Rate
            self.cbr_data = pd.read_csv(f"{data_path}Central Bank Rate (CBR)  .csv", skiprows=2)
            self.cbr_alt = pd.read_csv(f"{data_path}Central Bank Rates ().csv", skiprows=2)
            datasets_loaded += 2
            print("✅ Loaded Central Bank Rate Data")
        except Exception as e:
            print(f"❌ Could not load CBR Data: {e}")
            
        try:
            # Repo and Reverse Repo
            self.repo_data = pd.read_csv(f"{data_path}Repo and Reverse Repo .csv", skiprows=2)
            self.horizontal_repo = pd.read_csv(f"{data_path}Horizontal Repo Market.csv", skiprows=2)
            datasets_loaded += 2
            print("✅ Loaded Repo Market Data")
        except Exception as e:
            print(f"❌ Could not load Repo Data: {e}")
            
        try:
            # Discount Window
            self.discount_window = pd.read_csv(f"{data_path}Discount Window.csv", skiprows=2)
            datasets_loaded += 1
            print("✅ Loaded Discount Window Data")
        except Exception as e:
            print(f"❌ Could not load Discount Window Data: {e}")
            
        try:
            # Government Securities
            self.gov_securities = pd.read_csv(f"{data_path}Government Securities Auction and Maturities Schedule.csv", skiprows=2)
            self.treasury_bills = pd.read_csv(f"{data_path}Issues of Treasury Bills.csv", skiprows=2)
            self.treasury_bonds = pd.read_csv(f"{data_path}Issues of Treasury Bonds.csv", skiprows=2)
            datasets_loaded += 3
            print("✅ Loaded Government Securities Data")
        except Exception as e:
            print(f"❌ Could not load Government Securities Data: {e}")
            
        try:
            # Daily interbank activity
            self.daily_interbank = pd.read_csv(f"{data_path}Daily KES Interbank Activity Report.csv", skiprows=2)
            datasets_loaded += 1
            print("✅ Loaded Daily Interbank Activity Data")
        except Exception as e:
            print(f"❌ Could not load Daily Interbank Data: {e}")
        
        print(f"📊 Successfully loaded {datasets_loaded} datasets")
        return datasets_loaded > 0
    
    def clean_interbank_data(self):
        """Clean and standardize all interbank market data"""
        
        # List of all datasets to clean
        datasets = ['interbank_rates', 'interbank_rates_alt', 'cbr_data', 'cbr_alt', 
                   'repo_data', 'horizontal_repo', 'discount_window', 'gov_securities',
                   'treasury_bills', 'treasury_bonds', 'daily_interbank']
        
        for dataset_name in datasets:
            if hasattr(self, dataset_name):
                dataset = getattr(self, dataset_name)
                
                # Remove completely empty rows
                dataset = dataset.dropna(how='all')
                
                # Clean numeric columns
                for col in dataset.columns[1:]:  # Skip first column (usually date/period)
                    if dataset[col].dtype == 'object':
                        try:
                            # Handle common formatting issues
                            dataset[col] = dataset[col].astype(str).str.replace(',', '')
                            dataset[col] = dataset[col].str.replace('(', '-').str.replace(')', '')
                            dataset[col] = dataset[col].str.replace('%', '')
                            dataset[col] = pd.to_numeric(dataset[col], errors='coerce')
                        except:
                            pass
                
                # Handle date columns
                if len(dataset.columns) > 0:
                    first_col = dataset.columns[0]
                    if any(keyword in first_col.lower() for keyword in ['date', 'period', 'time']):
                        try:
                            dataset[first_col] = pd.to_datetime(dataset[first_col], errors='coerce')
                            dataset = dataset.set_index(first_col)
                        except:
                            pass
                
                setattr(self, dataset_name, dataset)
        
        return self
    
    def analyze_cbr_policy_impact(self):
        """Analyze Central Bank Rate policy impact on money markets"""
        
        if not hasattr(self, 'cbr_data') or self.cbr_data is None:
            print("CBR data not available")
            return None
        
        cbr_analysis = {}
        df = self.cbr_data.copy()
        
        # Find CBR column
        cbr_cols = [col for col in df.columns if any(keyword in col.lower() 
                   for keyword in ['cbr', 'central', 'bank', 'rate', 'policy'])]
        
        if cbr_cols:
            cbr_col = cbr_cols[0]
            cbr_series = df[cbr_col].dropna()
            
            if len(cbr_series) > 1:
                # CBR statistics
                cbr_analysis['cbr_stats'] = {
                    'current_cbr': cbr_series.iloc[-1],
                    'average_cbr': cbr_series.mean(),
                    'cbr_volatility': cbr_series.std(),
                    'max_cbr': cbr_series.max(),
                    'min_cbr': cbr_series.min(),
                    'cbr_range': cbr_series.max() - cbr_series.min()
                }
                
                # Policy changes analysis
                cbr_changes = cbr_series.diff().dropna()
                increases = cbr_changes[cbr_changes > 0]
                decreases = cbr_changes[cbr_changes < 0]
                
                cbr_analysis['policy_changes'] = {
                    'total_changes': len(cbr_changes[cbr_changes != 0]),
                    'increases': len(increases),
                    'decreases': len(decreases),
                    'average_increase': increases.mean() if len(increases) > 0 else 0,
                    'average_decrease': decreases.mean() if len(decreases) > 0 else 0,
                    'largest_increase': increases.max() if len(increases) > 0 else 0,
                    'largest_decrease': decreases.min() if len(decreases) > 0 else 0
                }
                
                # Policy stance analysis
                recent_cbr = cbr_series.tail(6)  # Last 6 observations
                cbr_trend = np.polyfit(range(len(recent_cbr)), recent_cbr, 1)[0]
                
                if cbr_trend > 0.1:
                    stance = "Tightening"
                elif cbr_trend < -0.1:
                    stance = "Easing"
                else:
                    stance = "Neutral"
                
                cbr_analysis['policy_stance'] = {
                    'current_stance': stance,
                    'trend_slope': cbr_trend,
                    'recent_average': recent_cbr.mean()
                }
        
        return cbr_analysis
    
    def analyze_interbank_liquidity(self):
        """Analyze interbank market liquidity conditions"""
        
        liquidity_analysis = {}
        
        # Interbank rates analysis
        if hasattr(self, 'interbank_rates'):
            df = self.interbank_rates.select_dtypes(include=[np.number])
            
            if len(df.columns) > 0:
                # Rate volatility indicates liquidity stress
                rate_volatility = df.std()
                volume_columns = [col for col in df.columns if 'volume' in col.lower()]
                rate_columns = [col for col in df.columns if 'rate' in col.lower()]
                
                liquidity_analysis['rate_volatility'] = {
                    'most_volatile_rate': rate_volatility.idxmax(),
                    'least_volatile_rate': rate_volatility.idxmin(),
                    'average_volatility': rate_volatility.mean(),
                    'volatility_range': rate_volatility.max() - rate_volatility.min()
                }
                
                # Volume analysis
                if volume_columns:
                    volumes = df[volume_columns].mean()
                    liquidity_analysis['volume_metrics'] = {
                        'average_daily_volume': volumes.mean(),
                        'volume_trend': np.polyfit(range(len(df)), df[volume_columns].sum(axis=1), 1)[0],
                        'peak_volume': df[volume_columns].sum(axis=1).max(),
                        'low_volume': df[volume_columns].sum(axis=1).min()
                    }
        
        # Repo market analysis
        if hasattr(self, 'repo_data'):
            repo_df = self.repo_data.select_dtypes(include=[np.number])
            
            if len(repo_df.columns) > 0:
                liquidity_analysis['repo_activity'] = {
                    'average_repo_rate': repo_df.mean().mean(),
                    'repo_volatility': repo_df.std().mean(),
                    'repo_trend': np.polyfit(range(len(repo_df)), repo_df.mean(axis=1), 1)[0]
                }
        
        # Discount window usage
        if hasattr(self, 'discount_window'):
            dw_df = self.discount_window.select_dtypes(include=[np.number])
            
            if len(dw_df.columns) > 0:
                total_usage = dw_df.sum(axis=1)
                liquidity_analysis['discount_window'] = {
                    'average_usage': total_usage.mean(),
                    'peak_usage': total_usage.max(),
                    'usage_frequency': (total_usage > 0).sum(),
                    'stress_indicator': total_usage.std()
                }
        
        return liquidity_analysis
    
    def analyze_government_securities_market(self):
        """Analyze government securities auction and market dynamics"""
        
        if not hasattr(self, 'treasury_bills') and not hasattr(self, 'treasury_bonds'):
            print("Government securities data not available")
            return None
        
        securities_analysis = {}
        
        # Treasury Bills analysis
        if hasattr(self, 'treasury_bills'):
            tb_df = self.treasury_bills.select_dtypes(include=[np.number])
            
            if len(tb_df.columns) > 0:
                securities_analysis['treasury_bills'] = {
                    'average_issue_size': tb_df.mean().mean(),
                    'total_outstanding': tb_df.sum().sum(),
                    'issuance_trend': np.polyfit(range(len(tb_df)), tb_df.sum(axis=1), 1)[0],
                    'market_activity': tb_df.std().mean()
                }
        
        # Treasury Bonds analysis
        if hasattr(self, 'treasury_bonds'):
            bond_df = self.treasury_bonds.select_dtypes(include=[np.number])
            
            if len(bond_df.columns) > 0:
                securities_analysis['treasury_bonds'] = {
                    'average_issue_size': bond_df.mean().mean(),
                    'total_outstanding': bond_df.sum().sum(),
                    'issuance_trend': np.polyfit(range(len(bond_df)), bond_df.sum(axis=1), 1)[0],
                    'market_depth': bond_df.std().mean()
                }
        
        # Calculate yield curve characteristics if rate data available
        rate_cols = []
        if hasattr(self, 'treasury_bills'):
            rate_cols.extend([col for col in self.treasury_bills.columns if 'rate' in col.lower() or 'yield' in col.lower()])
        if hasattr(self, 'treasury_bonds'):
            rate_cols.extend([col for col in self.treasury_bonds.columns if 'rate' in col.lower() or 'yield' in col.lower()])
        
        if rate_cols:
            securities_analysis['yield_curve'] = {
                'curve_steepness': 'Analysis requires maturity-specific data',
                'average_short_rate': 'Calculated from available bill rates',
                'average_long_rate': 'Calculated from available bond rates'
            }
        
        return securities_analysis
    
    def calculate_market_stress_indicators(self):
        """Calculate various market stress and liquidity indicators"""
        
        stress_indicators = {}
        
        # Rate spread analysis
        if hasattr(self, 'interbank_rates'):
            rates_df = self.interbank_rates.select_dtypes(include=[np.number])
            
            if len(rates_df.columns) >= 2:
                # Calculate spreads between different tenors
                rate_spreads = rates_df.max(axis=1) - rates_df.min(axis=1)
                
                stress_indicators['rate_spreads'] = {
                    'average_spread': rate_spreads.mean(),
                    'max_spread': rate_spreads.max(),
                    'spread_volatility': rate_spreads.std(),
                    'current_spread': rate_spreads.iloc[-1] if len(rate_spreads) > 0 else None
                }
        
        # Volume concentration analysis
        if hasattr(self, 'daily_interbank'):
            daily_df = self.daily_interbank.select_dtypes(include=[np.number])
            
            if len(daily_df.columns) > 0:
                volume_concentration = daily_df.std() / daily_df.mean()
                
                stress_indicators['volume_concentration'] = {
                    'concentration_index': volume_concentration.mean(),
                    'market_fragmentation': volume_concentration.std(),
                    'liquidity_distribution': 'Based on volume concentration metrics'
                }
        
        # Discount window stress indicator
        if hasattr(self, 'discount_window'):
            dw_df = self.discount_window.select_dtypes(include=[np.number])
            
            if len(dw_df.columns) > 0:
                dw_usage = dw_df.sum(axis=1)
                dw_stress = (dw_usage > dw_usage.quantile(0.75)).astype(int)
                
                stress_indicators['discount_window_stress'] = {
                    'stress_periods': dw_stress.sum(),
                    'stress_frequency': dw_stress.mean(),
                    'recent_stress': dw_stress.tail(10).mean()
                }
        
        return stress_indicators
    
    def create_interbank_visualizations(self):
        """Create comprehensive interbank market visualizations"""
        
        visualizations = {}
        
        # 1. CBR Policy Rate Chart
        if hasattr(self, 'cbr_data'):
            cbr_df = self.cbr_data.copy()
            cbr_cols = [col for col in cbr_df.columns if any(keyword in col.lower() 
                       for keyword in ['cbr', 'central', 'rate'])]
            
            if cbr_cols and hasattr(cbr_df, 'index') and cbr_df.index.dtype.kind in ['M', 'O']:
                fig_cbr = go.Figure()
                
                cbr_series = cbr_df[cbr_cols[0]].dropna()
                
                fig_cbr.add_trace(go.Scatter(
                    x=cbr_series.index,
                    y=cbr_series,
                    mode='lines+markers',
                    name='Central Bank Rate',
                    line=dict(color='darkblue', width=3),
                    marker=dict(size=8)
                ))
                
                # Add trend line
                if len(cbr_series) > 1:
                    x_numeric = np.arange(len(cbr_series))
                    trend_coef = np.polyfit(x_numeric, cbr_series, 1)
                    trend_line = np.poly1d(trend_coef)(x_numeric)
                    
                    fig_cbr.add_trace(go.Scatter(
                        x=cbr_series.index,
                        y=trend_line,
                        mode='lines',
                        name='Trend',
                        line=dict(color='red', dash='dash')
                    ))
                
                fig_cbr.update_layout(
                    title='Central Bank Rate (CBR) Policy Trend',
                    xaxis_title='Date',
                    yaxis_title='Rate (%)',
                    template='plotly_white',
                    height=500
                )
                
                visualizations['cbr_policy'] = fig_cbr
        
        # 2. Interbank Rates and Volumes
        if hasattr(self, 'interbank_rates'):
            rates_df = self.interbank_rates.select_dtypes(include=[np.number])
            
            if len(rates_df.columns) >= 2 and hasattr(rates_df, 'index'):
                fig_interbank = make_subplots(specs=[[{"secondary_y": True}]])
                
                # Rates
                rate_cols = [col for col in rates_df.columns if 'rate' in col.lower()][:3]
                colors = ['blue', 'green', 'purple']
                
                for i, col in enumerate(rate_cols):
                    if not rates_df[col].isna().all():
                        fig_interbank.add_trace(
                            go.Scatter(x=rates_df.index, y=rates_df[col], 
                                     name=col, line=dict(color=colors[i % len(colors)])),
                            secondary_y=False
                        )
                
                # Volumes
                volume_cols = [col for col in rates_df.columns if 'volume' in col.lower()][:2]
                for col in volume_cols:
                    if not rates_df[col].isna().all():
                        fig_interbank.add_trace(
                            go.Bar(x=rates_df.index, y=rates_df[col], 
                                  name=col, opacity=0.3),
                            secondary_y=True
                        )
                
                fig_interbank.update_layout(
                    title='Interbank Rates and Trading Volumes',
                    template='plotly_white',
                    height=600
                )
                fig_interbank.update_yaxes(title_text="Interest Rate (%)", secondary_y=False)
                fig_interbank.update_yaxes(title_text="Volume (KES Millions)", secondary_y=True)
                
                visualizations['interbank_activity'] = fig_interbank
        
        # 3. Government Securities Issuance
        if hasattr(self, 'treasury_bills') and hasattr(self, 'treasury_bonds'):
            tb_df = self.treasury_bills.select_dtypes(include=[np.number])
            bond_df = self.treasury_bonds.select_dtypes(include=[np.number])
            
            if len(tb_df.columns) > 0 and len(bond_df.columns) > 0:
                fig_securities = go.Figure()
                
                # Treasury Bills
                tb_total = tb_df.sum(axis=1)
                fig_securities.add_trace(go.Bar(
                    x=tb_total.index,
                    y=tb_total,
                    name='Treasury Bills',
                    marker_color='lightblue'
                ))
                
                # Treasury Bonds
                bond_total = bond_df.sum(axis=1)
                fig_securities.add_trace(go.Bar(
                    x=bond_total.index,
                    y=bond_total,
                    name='Treasury Bonds',
                    marker_color='darkblue'
                ))
                
                fig_securities.update_layout(
                    title='Government Securities Issuance',
                    xaxis_title='Period',
                    yaxis_title='Amount Issued (KES Millions)',
                    barmode='stack',
                    template='plotly_white',
                    height=500
                )
                
                visualizations['securities_issuance'] = fig_securities
        
        # 4. Market Stress Indicators
        stress_data = self.calculate_market_stress_indicators()
        if stress_data and 'rate_spreads' in stress_data:
            # Create a stress indicator dashboard
            fig_stress = make_subplots(
                rows=2, cols=2,
                subplot_titles=['Rate Spreads', 'Volume Concentration', 
                               'Discount Window Usage', 'Market Liquidity'],
                specs=[[{"type": "scatter"}, {"type": "bar"}],
                       [{"type": "scatter"}, {"type": "indicator"}]]
            )
            
            # Add placeholder visualizations for stress indicators
            # This would be populated with actual stress metrics
            fig_stress.update_layout(
                title='Money Market Stress Indicators Dashboard',
                template='plotly_white',
                height=600
            )
            
            visualizations['stress_dashboard'] = fig_stress
        
        return visualizations
    
    def generate_interbank_insights(self):
        """Generate key insights from interbank market analysis"""
        
        insights = []
        
        # CBR policy insights
        cbr_analysis = self.analyze_cbr_policy_impact()
        if cbr_analysis and 'policy_stance' in cbr_analysis:
            stance = cbr_analysis['policy_stance']['current_stance']
            current_cbr = cbr_analysis['cbr_stats']['current_cbr']
            
            insights.append(f"Current monetary policy stance is {stance} with CBR at {current_cbr:.2f}%")
            
            if 'policy_changes' in cbr_analysis:
                changes = cbr_analysis['policy_changes']
                total_changes = changes['total_changes']
                increases = changes['increases']
                decreases = changes['decreases']
                
                if increases > decreases:
                    insights.append(f"CBR has been raised {increases} times vs {decreases} cuts, indicating tightening bias")
                elif decreases > increases:
                    insights.append(f"CBR has been cut {decreases} times vs {increases} hikes, showing easing bias")
                else:
                    insights.append("Balanced monetary policy with equal number of rate hikes and cuts")
        
        # Liquidity insights
        liquidity_analysis = self.analyze_interbank_liquidity()
        if liquidity_analysis:
            if 'rate_volatility' in liquidity_analysis:
                vol_data = liquidity_analysis['rate_volatility']
                avg_vol = vol_data['average_volatility']
                
                if avg_vol > 2:
                    insights.append(f"High interbank rate volatility ({avg_vol:.2f}%) indicates liquidity stress")
                elif avg_vol > 1:
                    insights.append(f"Moderate interbank rate volatility ({avg_vol:.2f}%) shows normal market conditions")
                else:
                    insights.append(f"Low interbank rate volatility ({avg_vol:.2f}%) indicates ample liquidity")
            
            if 'discount_window' in liquidity_analysis:
                dw_data = liquidity_analysis['discount_window']
                usage_freq = dw_data.get('usage_frequency', 0)
                
                if usage_freq > 10:
                    insights.append("Frequent discount window usage suggests persistent liquidity challenges")
                elif usage_freq > 5:
                    insights.append("Moderate discount window usage indicates occasional liquidity needs")
                else:
                    insights.append("Low discount window usage shows healthy liquidity conditions")
        
        # Government securities market insights
        securities_analysis = self.analyze_government_securities_market()
        if securities_analysis:
            if 'treasury_bills' in securities_analysis:
                tb_trend = securities_analysis['treasury_bills']['issuance_trend']
                if tb_trend > 0:
                    insights.append("Treasury bill issuance is trending upward, indicating growing funding needs")
                else:
                    insights.append("Treasury bill issuance is stable or declining")
            
            if 'treasury_bonds' in securities_analysis:
                bond_trend = securities_analysis['treasury_bonds']['issuance_trend']
                if bond_trend > 0:
                    insights.append("Treasury bond issuance is increasing, showing long-term funding requirements")
        
        return insights

# Initialize the analyzer
analyzer = InterbankMarketsAnalyzer()
print("🏦 Interbank Markets Analyzer Initialized")

In [ ]:
# Load and Clean Data
print("📊 Loading interbank market data...")
if analyzer.load_interbank_data():
    analyzer.clean_interbank_data()
    print("✅ Data loaded and cleaned successfully")
else:
    print("❌ Failed to load sufficient data")

In [ ]:
# Analyze CBR Policy Impact
print("\n🏛️ Analyzing Central Bank Rate policy impact...")
cbr_analysis = analyzer.analyze_cbr_policy_impact()

if cbr_analysis:
    print("\n📊 CBR Policy Analysis:")
    
    if 'cbr_stats' in cbr_analysis:
        stats = cbr_analysis['cbr_stats']
        print(f"  Current CBR: {stats['current_cbr']:.2f}%")
        print(f"  Average CBR: {stats['average_cbr']:.2f}%")
        print(f"  CBR Range: {stats['min_cbr']:.2f}% - {stats['max_cbr']:.2f}%")
    
    if 'policy_stance' in cbr_analysis:
        stance_data = cbr_analysis['policy_stance']
        print(f"  Current Policy Stance: {stance_data['current_stance']}")
        print(f"  Trend Direction: {stance_data['trend_slope']:.3f}")

In [ ]:
# Analyze Interbank Liquidity
print("\n💧 Analyzing interbank liquidity conditions...")
liquidity_analysis = analyzer.analyze_interbank_liquidity()

if liquidity_analysis:
    print("\n🔍 Liquidity Analysis Results:")
    
    if 'rate_volatility' in liquidity_analysis:
        vol_data = liquidity_analysis['rate_volatility']
        print(f"  Most Volatile Rate: {vol_data['most_volatile_rate']}")
        print(f"  Average Volatility: {vol_data['average_volatility']:.3f}%")
    
    if 'volume_metrics' in liquidity_analysis:
        vol_metrics = liquidity_analysis['volume_metrics']
        print(f"  Average Daily Volume: {vol_metrics['average_daily_volume']:.0f} KES Millions")
        print(f"  Volume Trend: {'Increasing' if vol_metrics['volume_trend'] > 0 else 'Decreasing'}")
    
    if 'discount_window' in liquidity_analysis:
        dw_data = liquidity_analysis['discount_window']
        print(f"  Discount Window Usage Frequency: {dw_data['usage_frequency']} periods")
        print(f"  Average Usage: {dw_data['average_usage']:.0f} KES Millions")

In [ ]:
# Analyze Government Securities Market
print("\n🏛️ Analyzing government securities market...")
securities_analysis = analyzer.analyze_government_securities_market()

if securities_analysis:
    print("\n📊 Government Securities Analysis:")
    
    if 'treasury_bills' in securities_analysis:
        tb_data = securities_analysis['treasury_bills']
        print(f"  Treasury Bills Average Issue: {tb_data['average_issue_size']:.0f} KES Millions")
        print(f"  T-Bills Trend: {'Increasing' if tb_data['issuance_trend'] > 0 else 'Decreasing'}")
    
    if 'treasury_bonds' in securities_analysis:
        bond_data = securities_analysis['treasury_bonds']
        print(f"  Treasury Bonds Average Issue: {bond_data['average_issue_size']:.0f} KES Millions")
        print(f"  T-Bonds Trend: {'Increasing' if bond_data['issuance_trend'] > 0 else 'Decreasing'}")

In [ ]:
# Calculate Market Stress Indicators
print("\n⚠️ Calculating market stress indicators...")
stress_indicators = analyzer.calculate_market_stress_indicators()

if stress_indicators:
    print("\n🚨 Market Stress Analysis:")
    
    if 'rate_spreads' in stress_indicators:
        spread_data = stress_indicators['rate_spreads']
        print(f"  Average Rate Spread: {spread_data['average_spread']:.3f}%")
        print(f"  Maximum Spread: {spread_data['max_spread']:.3f}%")
        print(f"  Current Spread: {spread_data['current_spread']:.3f}%")
    
    if 'discount_window_stress' in stress_indicators:
        dw_stress = stress_indicators['discount_window_stress']
        print(f"  Stress Periods: {dw_stress['stress_periods']}")
        print(f"  Stress Frequency: {dw_stress['stress_frequency']:.1%}")

In [ ]:
# Create Visualizations
print("\n🎨 Creating interbank market visualizations...")
visualizations = analyzer.create_interbank_visualizations()

print(f"✅ Generated {len(visualizations)} visualizations:")
for viz_name in visualizations.keys():
    print(f"  - {viz_name}")

# Display visualizations (uncomment in actual notebook)
# for name, fig in visualizations.items():
#     fig.show()

In [ ]:
# Generate Key Insights
print("\n💡 Generating interbank market insights...")
insights = analyzer.generate_interbank_insights()

print("\n🔍 Key Interbank Market Insights:")
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

## Interbank Markets Analysis Summary

This comprehensive interbank markets analysis provides:

### 🏛️ **Central Bank Policy Analysis**
- CBR policy rate trends and changes
- Monetary policy stance assessment
- Policy transmission effectiveness
- Historical rate cycle analysis

### 💧 **Liquidity Conditions**
- Interbank lending rates and spreads
- Trading volumes and market activity
- Repo and reverse repo operations
- Discount window usage patterns

### 📊 **Government Securities Market**
- Treasury bills and bonds issuance
- Auction dynamics and demand patterns
- Yield curve characteristics
- Market depth and liquidity

### ⚠️ **Market Stress Indicators**
- Rate volatility measures
- Liquidity concentration metrics
- Emergency funding usage
- Market fragmentation analysis

### 📈 **Key Insights Generated**
- Monetary policy transmission assessment
- Liquidity stress identification
- Market efficiency evaluation
- Risk concentration monitoring

### 🎯 **Applications**
This analysis supports:
- **Monetary Policy** decision making
- **Financial Stability** monitoring
- **Market Operations** optimization
- **Risk Management** strategies

The insights help central bankers, commercial banks, and market participants understand:
- Current liquidity conditions
- Policy effectiveness
- Market stress levels
- Operational efficiency opportunities